In [95]:
import json
import re

In [96]:
test_string='(ORDER (PIZZAORDER (NOT (TOPPING MUSHROOMS ) ) (NUMBER 1 ) (SIZE MEDIUM ) (TOPPING OLIVES ) (TOPPING ONIONS ) ) )'

In [97]:
def parse_to_dict(s):
    def parse(tokens):
        token = tokens.pop(0)
        if token == '(':
            key = tokens.pop(0)
            nested = {}
            while len(tokens) > 0 and tokens[0] != ')':
                if tokens[0] == '(':
                    temp_dict=parse(tokens)
                    for k,v in temp_dict.items():
                        if k in nested:
                            nested[k].append(v)
                        elif k == 'DRINKORDER' or k == 'PIZZAORDER' or k =='TOPPING':
                            nested[k]=[v]
                        else:
                            nested[k] = v
                else:
                    sub_key = tokens.pop(0)
                    if tokens[0] == ')':
                        tokens.pop(0)
                        return {key: sub_key}
            if len(tokens) > 0:
                tokens.pop(0)  
            return {key: nested}
        elif token == ')':
            return {}

    # Tokenize the string
    tokens = re.findall(r'\(|\)|\w+', s)
    return parse(tokens)

In [98]:
result = parse_to_dict(test_string)
print(result)

{'ORDER': {'PIZZAORDER': [{'NOT': {'TOPPING': ['MUSHROOMS']}, 'NUMBER': '1', 'SIZE': 'MEDIUM', 'TOPPING': ['OLIVES', 'ONIONS']}]}}


In [99]:
def normalize_dict_values(data):
    def normalize(data):
        if isinstance(data, dict):
            normalized_data = {}
            for key, value in data.items():
                if key in ['TOPPING', 'PIZZAORDER', 'DRINKORDER']:
                    normalized_data[key] = normalize(value)  # Keep these as lists
                elif isinstance(value, list) and len(value) == 1:
                    normalized_data[key] = normalize(value[0])  # Flatten single-item lists
                else:
                    normalized_data[key] = normalize(value)
            return normalized_data
        else:
            return data

    return normalize(data)

In [100]:
normalized_data = normalize_dict_values(result)
print(normalized_data)

{'ORDER': {'PIZZAORDER': [{'NOT': {'TOPPING': ['MUSHROOMS']}, 'NUMBER': '1', 'SIZE': 'MEDIUM', 'TOPPING': ['OLIVES', 'ONIONS']}]}}


In [101]:
def normalize_dict_values(data):
    def normalize(data):
        if isinstance(data, dict):
            normalized_data = {}
            for key, value in data.items():
                if isinstance(value, list) and len(value) == 1 and key not in ['TOPPING', 'PIZZAORDER', 'DRINKORDER']:
                    # Convert single-item lists to direct values
                    normalized_data[key] = normalize(value[0])
                else:
                    normalized_data[key] = normalize(value)
            return normalized_data
        else:
            return data

    return normalize(data)

In [102]:
json_string = json.dumps(result)
print(json_string)

{"ORDER": {"PIZZAORDER": [{"NOT": {"TOPPING": ["MUSHROOMS"]}, "NUMBER": "1", "SIZE": "MEDIUM", "TOPPING": ["OLIVES", "ONIONS"]}]}}
